In [20]:
import pandas as pd
import os

# Set the working directory to the project root
project_root = "/Users/pavelmakarchuk/salt-amt-calculator"
os.chdir(project_root)

# Load the 2026 and 2035 CSV files
df_2026 = pd.read_csv("nationwide_impacts/data/impacts_2026_2035/impacts_2026.csv")
df_2035 = pd.read_csv("nationwide_impacts/data/impacts_2026_2035/impacts_2035.csv")

# Filter to only get behavioral response reforms
reforms_2026 = df_2026[df_2026["reform"].str.contains("behavioral_responses_yes")].copy()
reforms_2035 = df_2035[df_2035["reform"].str.contains("behavioral_responses_yes")].copy()

print(f"Number of behavioral reforms in 2026: {len(reforms_2026)}")
print(f"Number of behavioral reforms in 2035: {len(reforms_2035)}")

# Create base reform names (removing both types of suffixes)
reforms_2026["reform_base"] = reforms_2026["reform"].str.replace("_vs_current_policy", "", regex=False)
reforms_2026["reform_base"] = reforms_2026["reform_base"].str.replace("_vs_current_law", "", regex=False)
reforms_2035["reform_base"] = reforms_2035["reform"].str.replace("_year_2035_vs_current_policy", "", regex=False)
reforms_2035["reform_base"] = reforms_2035["reform_base"].str.replace("_year_2035_vs_current_law", "", regex=False)

# Create output directory for interpolated files
output_dir = "nationwide_impacts/data/interpolated_behavioral_responses"
os.makedirs(output_dir, exist_ok=True)

# Dictionary to map baseline values to correct suffix
suffix_map = {
    "Current Law": "_vs_current_law",
    "Current Policy": "_vs_current_policy"
}

# Loop through years 2027 to 2034
for year in range(2027, 2035):
    # Create merged dataframe with values from both years
    merged_year = pd.merge(
        reforms_2026[["reform_base", "total_income_change", "baseline"]].rename(columns={"total_income_change": "value_2026"}),
        reforms_2035[["reform_base", "total_income_change", "baseline"]].rename(columns={"total_income_change": "value_2035"}),
        on=["reform_base", "baseline"]
    )
    
    # Calculate weights for interpolation
    weight_2035 = (year - 2026) / (2035 - 2026)
    weight_2026 = 1 - weight_2035
    
    # Interpolate the values
    merged_year["total_income_change"] = (
        weight_2026 * merged_year["value_2026"] +
        weight_2035 * merged_year["value_2035"]
    )
    
    # Create the reform names for this year with correct suffixes
    merged_year["reform"] = merged_year.apply(
        lambda x: x["reform_base"] + f"_year_{year}" + suffix_map[x["baseline"]], 
        axis=1
    )
    
    # Create the output dataframe
    output_df = pd.DataFrame({
        "reform": merged_year["reform"],
        "baseline": merged_year["baseline"],
        "year": year,
        "total_income_change": merged_year["total_income_change"]
    })
    
    # Save to CSV
    output_file = os.path.join(output_dir, f"behavioral_responses_{year}.csv")
    output_df.to_csv(output_file, index=False)
    print(f"Saved {output_file} with {len(output_df)} reforms")
    
    # Print sample of first file to verify format
    if year == 2027:
        print("\nSample of 2027 reforms:")
        print(output_df[["reform", "baseline"]].head())

print("\nInterpolation complete! Files saved in:", output_dir)

Number of behavioral reforms in 2026: 264
Number of behavioral reforms in 2035: 264
Saved nationwide_impacts/data/interpolated_behavioral_responses/behavioral_responses_2027.csv with 264 reforms

Sample of 2027 reforms:
                                              reform        baseline
0  salt_0_cap_amt_pre_tcja_ex_pre_tcja_po_behavio...     Current Law
1  salt_0_cap_amt_pre_tcja_ex_pre_tcja_po_behavio...  Current Policy
2  salt_0_cap_amt_pre_tcja_ex_pre_tcja_po_behavio...     Current Law
3  salt_0_cap_amt_pre_tcja_ex_pre_tcja_po_behavio...  Current Policy
4  salt_0_cap_amt_pre_tcja_ex_tcja_po_behavioral_...     Current Law
Saved nationwide_impacts/data/interpolated_behavioral_responses/behavioral_responses_2028.csv with 264 reforms
Saved nationwide_impacts/data/interpolated_behavioral_responses/behavioral_responses_2029.csv with 264 reforms
Saved nationwide_impacts/data/interpolated_behavioral_responses/behavioral_responses_2030.csv with 264 reforms
Saved nationwide_impacts/data/int